# WIP, ignore this notebook

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from torch.utils.data import Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, fbeta_score, classification_report

from PIL import Image
import numpy as np
import os

import torch
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms

from sklearn.metrics import classification_report, fbeta_score

from tqdm import tqdm
import torch.nn.functional as F

In [13]:
# Define hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 0.0005

In [12]:
# Set the device
device = "cpu"
if torch.cuda.is_available():
    device =  "cuda"
elif torch.backends.mps.is_available():
    device = "mps" # Use M1 Mac GPU
    device = "cpu"
print(device)

cpu


In [7]:
def read_from_csv(filename):
    df = pd.read_csv(filename)
    data = [(str(row['Text']), row['Label']) for _, row in df.iterrows() if len(str(row['Text'])) >= 3 and str(row['Text']) != ""]
    return data

# Example usage:
train_list= read_from_csv('../../../datasets/extractor_classifier/train_data_text.csv')
validation_list = read_from_csv('../../../datasets/extractor_classifier/validation_data_text.csv')
test_list = read_from_csv('../../../datasets/extractor_classifier/test_data_text.csv')

In [8]:
class ExtractorClassifierDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self. data[idx]
        label_tensor = torch.zeros(2)
        label_tensor[label] = 1
        return text, label_tensor

# Define the train/validation/test datasets
train_data = ExtractorClassifierDataset(train_list)
validation_data = ExtractorClassifierDataset(validation_list)
test_data = ExtractorClassifierDataset(test_list)

In [14]:
# Define the dataloaders for each dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, )
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
def train(model, model_path, tokenizer, train_loader, validation_loader, test_loader, criterion, optimizer, epochs):

    model.to(device)

    # Define variables to track the best validation accuracy and the corresponding model state
    best_valid_f_beta = 0.0
    train_loss_values = []
    validation_loss_values = []
    train_f_beta_values = []
    validation_f_beta_values = []

    train_true_labels = []
    train_predicted_labels = []

    for epoch in range(epochs):
        # Train the model for one epoch
        train_loss = 0.0
        model.train()
        for texts, labels in tqdm(train_loader):
            inputs = tokenizer.batch_encode_plus(texts, return_tensors="pt", truncation=True, padding=True)
            inputs = inputs.to(device)
            labels = labels.to(device).to(torch.float32)

            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            train_loss += loss.item() * len(texts)
            loss.backward()
            optimizer.step()

            outputs = F.softmax(outputs.logits, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            train_predicted_labels.extend(predicted.detach().cpu())
            train_true_labels.extend(labels.detach().cpu())

        train_loss = train_loss / len(train_loader.dataset)
        train_loss_values.append(train_loss)

        train_f_beta = fbeta_score(train_true_labels, train_predicted_labels, average="binary", beta=0.5)
        train_f_beta_values.append(train_f_beta)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss:.4f} - Train Fbeta: {train_f_beta:.4f}")

        # Evaluate the model on the validation set
        validation_loss, validation_f_beta = validate_model(model, validation_loader, tokenizer)
        validation_loss_values.append(validation_loss)
        validation_f_beta_values.append(validation_f_beta)

        # Save the model state if the current validation accuracy is better than the previous best
        if validation_f_beta > best_valid_f_beta:
            best_valid_f_beta = validation_f_beta
            print(f"Updated best model: epoch {epoch+1}")
            model.save_pretrained(model_path)

    # Load the best model state and evaluate on the test set
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Test model:")
    test_loss, test_f_beta = validate_model(model, test_loader, tokenizer)
    print(f"Test Loss: {test_loss:.4f} - Test Fbeta: {test_f_beta:.4f}")

    return model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values

def validate_model(model, data_loader, tokenizer):
    model.to(device)
    model.eval()

    valid_loss = 0.0
    valid_true_labels = []
    valid_predicted_labels = []

    with torch.no_grad():
        for texts, labels in data_loader:
            inputs = tokenizer.batch_encode_plus(texts, return_tensors="pt", truncation=True, padding=True)
            inputs = inputs.to(device)
            labels = labels.to(device).to(torch.float32)
            outputs = model(**inputs)

            loss = criterion(outputs.logits, labels)
            valid_loss += loss.item() * len(texts)

            outputs = F.softmax(outputs.logits, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            valid_predicted_labels.extend(predicted.detach().cpu())
            valid_true_labels.extend(labels.detach().cpu())

    f_beta_score = fbeta_score(valid_true_labels, valid_predicted_labels, average="binary", beta=0.5)
    valid_loss = valid_loss / len(data_loader.dataset)

    print(f"Valid Loss: {valid_loss:.4f} - Valid Fbeta: {f_beta_score:.4f}")
    print(classification_report(valid_true_labels, valid_predicted_labels))
    return valid_loss, f_beta_score

In [15]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
model_path = "model_roberta"

if os.path.isfile(model_path):
    print("Evaluate model")
    model.from_pretrained(model_path)
    print("Validation data:")
    validate_model(model, validation_loader, tokenizer)
    print("Test data:")
    validate_model(model, test_loader, tokenizer)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(model, model_path, tokenizer, train_loader, validation_loader, test_loader, criterion, optimizer, 50)
    np.save("model_roberta/roberta_train_loss", train_loss_values)
    np.save("model_roberta/roberta_train_f_beta", train_f_beta_values)
    np.save("model_roberta/rroberta_valid_loss", validation_loss_values)
    np.save("model_roberta/roberta_valid_f_beta", validation_f_beta_values)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should pr

Train model


 22%|██▏       | 24/110 [05:10<18:31, 12.93s/it]


KeyboardInterrupt: 